# SQL query from table names

In This notebook we are going to test if using just the name of the table, and a shord definition of its contect we can use a model like GTP3.5-Turbo to select which tables are necessary to create a SQL Order to answer the user petition.

In [6]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [7]:
#Functio to call the model.
def return_OAI(user_message):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)
    context = []
    context.append({'role':'system', "content": user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=context,
            temperature=0,
        )

    return (response.choices[0].message.content)

In [8]:
import pandas as pd
import json

# Step 1: Define the data for the tables and their definitions
data = {
    'table': ['employees', 'salary', 'studies'],
    'definition': [
        'Employee information',
        'Salary details for each year',
        'Educational studies'
    ]
}

# Step 2: Create the DataFrame
df = pd.DataFrame(data)

# Step 3: Create a dictionary from the DataFrame for JSON output
tables_dict = {row['table']: row['definition'] for index, row in df.iterrows()}

# Step 4: Convert the dictionary to a JSON string
tables_json = json.dumps({"tables": tables_dict}, indent=4)

# Step 5: Print the JSON output
print(tables_json)

{
    "tables": {
        "employees": "Employee information",
        "salary": "Salary details for each year",
        "studies": "Educational studies"
    }
}


In [9]:
import json

text_tables = '\n'.join([f"{row['table']}: {row['definition']}" for index, row in df.iterrows()])
tables_json = json.dumps({"tables": text_tables}, indent=4)


In [10]:
print(text_tables)

employees: Employee information
salary: Salary details for each year
studies: Educational studies


In [11]:
user_question = "What information do we have about employee salaries?"

# Step 6: Define the prompt using the formatted tables JSON and user question
prompt_question_tables = """
Given the following tables and their content definitions,
### Tables
{tables}

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a JSON format.
### User Question:
{question}
"""

# Step 7: Format the prompt with the tables JSON and user question
formatted_prompt = prompt_question_tables.format(tables=tables_dict, question=user_question)

# Step 8: Print the formatted prompt
print(formatted_prompt)


Given the following tables and their content definitions,
### Tables
{'employees': 'Employee information', 'salary': 'Salary details for each year', 'studies': 'Educational studies'}

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a JSON format.
### User Question:
What information do we have about employee salaries?



In [12]:
#Creating the prompt, with the user questions and the tables definitions.
pqt1 = prompt_question_tables.format(tables=tables_dict, question=user_question)

In [13]:
print(return_OAI(pqt1))

```json
["employees", "salary"]
```


In [14]:
pqt3 = prompt_question_tables.format(tables=tables_json, question=user_question)


In [15]:
print(return_OAI(pqt3))

```json
{
    "tables": ["employees", "salary"]
}
```


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try a few versions if you have time
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

# version

In [16]:


data = {
    'table': ['FIFA', 'Super Mario Bros', 'Minecraft'],
    'definition': [
        'A popular football simulation game',
        'A classic platformer featuring Mario',
        'A sandbox game that allows for creativity and exploration'
    ],
    'price': [
        '$59.99',
        '$49.99',
        '$29.99'
    ],
    'type': [
        'Sports',
        'Platformer',
        'Sandbox'
    ]
}

df = pd.DataFrame(data)

tables_dict = {row['table']: row['definition'] for index, row in df.iterrows()}

tables_json = json.dumps({"tables": tables_dict}, indent=4)

print(tables_json)

{
    "tables": {
        "FIFA": "A popular football simulation game",
        "Super Mario Bros": "A classic platformer featuring Mario",
        "Minecraft": "A sandbox game that allows for creativity and exploration"
    }
}


In [17]:

user_question = "What information do we have about the games, their price and type?"

prompt_question_tables = """
Given the following tables and their content definitions,
### Tables
{tables}

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a JSON format.
### User Question:
{question}
"""

formatted_prompt = prompt_question_tables.format(tables=tables_dict, question=user_question)

print(formatted_prompt)


Given the following tables and their content definitions,
### Tables
{'FIFA': 'A popular football simulation game', 'Super Mario Bros': 'A classic platformer featuring Mario', 'Minecraft': 'A sandbox game that allows for creativity and exploration'}

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a JSON format.
### User Question:
What information do we have about the games, their price and type?



In [18]:
pqt1 = prompt_question_tables.format(tables=tables_dict, question=user_question)

In [19]:
pqt3 = prompt_question_tables.format(tables=tables_json, question=user_question)


In [20]:
print(return_OAI(pqt1))
print(return_OAI(pqt3))

```json
["Games"]
```
{
    "tables": {
        "FIFA": "A popular football simulation game",
        "Super Mario Bros": "A classic platformer featuring Mario"
    }
}


# ---------------------------------------------------------------------------------------------

# Report

## Data Structure 

### Employees
- **Data Attributes**:
- Employees
- Salaries
- Education

### Video Games
- **Data Attributes**:
- Game Titles
- Prices
- Types

## Successes
### Employees
- **Clarity**: The employee data structure provides a clear understanding of roles and compensation, facilitating HR management.
- **Ease of Use**: Employee data can be easily integrated into HR systems to track performance, salary adjustments, and training needs.

### Video Games
- **Interactivity**: The video game data structure effectively engages gamers by providing relevant information about game choices, improving decision-making.
- **Structured Information**: The structured format allows for easy integration with gaming platforms, facilitating user interaction and game selection.

## Disadvantages
### Employees
- **Complexity**: Including multiple attributes (such as education and roles) can complicate employee performance analysis or decision-making. - **Data Sensitivity**: Employee information is sensitive and must be handled carefully to ensure privacy.

###Video Games
- **Subjectivity**: Game descriptions can vary greatly in quality, leading to potential inconsistencies in how players perceive games.
- **Limited Context**: Video game data may lack depth regarding the player experience, as attributes focus primarily on core gameplay elements.

# Conclusion
A comparison of employee data and video game data highlights significant differences in structure and purpose. Employee data primarily focuses on roles and compensation, while video game data focuses on interaction and user experience. Both structures have strengths and weaknesses; employee data is clear and easy to use for HR purposes, while video game data enhances player engagement but can suffer from subjectivity and limited context. Understanding these differences is critical to effective HR management and developing engaging gaming experiences. Future efforts should focus on improving data attributes in both contexts to improve usability and relevance.